In [ ]:
pip install transformers

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import torch
import torch.nn as nn

import transformers
from transformers import BertTokenizer, BertForSequenceClassification

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip drive/MyDrive/MSR_data/MSR_Project_files.zip

Archive:  drive/MyDrive/MSR_data/MSR_Project_files.zip
replace __MACOSX/._MSR Project files ? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: __MACOSX/._MSR Project files   
  inflating: MSR Project files /VerifiedDataLabels.csv  
  inflating: __MACOSX/MSR Project files /._VerifiedDataLabels.csv  
  inflating: MSR Project files /url_df.csv  
  inflating: __MACOSX/MSR Project files /._url_df.csv  
  inflating: MSR Project files /.DS_Store  
  inflating: __MACOSX/MSR Project files /._.DS_Store  
  inflating: MSR Project files /data_tensor_fnl  A

  inflating: __MACOSX/MSR Project files /._data_tensor_fnl  
  inflating: MSR Project files /stem_sentence_corpus  
  inflating: __MACOSX/MSR Project files /._stem_sentence_corpus  
  inflating: MSR Project files /I_tensor  
  inflating: __MACOSX/MSR Project files /._I_tensor  
  inflating: MSR Project files /df_csv_joined.csv  
  inflating: __MACOSX/MSR Project files /._df_csv_joined.csv  
  inflating: MSR Project files /MSR_text_2.csv  
 

In [ ]:
!unzip drive/MyDrive/MSR_data/MSR_Project_files.zip > /dev/null

replace __MACOSX/._MSR Project files ? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
A


In [ ]:
url_df_read = pd.read_csv('drive/MyDrive/MSR_data/url_df.csv', index_col=[0])
url_df_read

,URL_link
0,https://dq06ugkuram52.cloudfront.net/files/637...
1,https://dq06ugkuram52.cloudfront.net/files/635...
2,https://dq06ugkuram52.cloudfront.net/files/637...
3,https://dq06ugkuram52.cloudfront.net/files/635...
4,https://dq06ugkuram52.cloudfront.net/files/635...
...,...
1867,https://dq06ugkuram52.cloudfront.net/files/730...
1868,https://dq06ugkuram52.cloudfront.net/files/730...
1869,https://dq06ugkuram52.cloudfront.net/files/730...
1870,https://dq06ugkuram52.cloudfront.net/files/730...


In [ ]:
MSR_df_read = pd.read_csv('drive/MyDrive/MSR_data/MSR_text_2.csv', index_col=[0])
MSR_df_read

,Text
0,10/19/2020\n\nSlavery and Human trafﬁcking | S...
1,\n\n \n\n \n\n \n\n \n\n \n\n \n\nModern Slav...
2,10/19/2020\n\nSlavery | Hall and Woodhouse\n\n...
3,Greene King | MODERN SLAVERY STATEMENT\n\nhttp...
4,Charles Wells Ltd \n \nModern Slavery and Huma...
...,...
1867,\n\n \nModern Slavery Act 2015 Statement \n...
1868,Upto 60% off + 20% off ends\n\n \n\n \n\nSearc...
1869,\n\nSlavery and Human Trafficking Transparenc...
1870,Reward Gateway Slavery\nand Human Trafficking\...


In [ ]:
df_csv_joined = pd.read_csv('drive/MyDrive/MSR_data/df_csv_joined.csv')
df_csv_joined

,Answer ID,Answer Page,Metric,Company,Value,Source Page,Text,URL_link
0,6382834,https://wikirate.org/~6382834,Walk Free Foundation+MSA whistleblowing mechan...,S A Brain Company Ltd,1,https://wikirate.org/~6375302,10/19/2020\n\nSlavery and Human trafﬁcking | S...,https://dq06ugkuram52.cloudfront.net/files/637...
1,6368402,https://wikirate.org/~6368402,Walk Free Foundation+MSA whistleblowing mechan...,Pension Protection Fund,1,https://wikirate.org/~6355100,\n\n \n\n \n\n \n\n \n\n \n\n \n\nModern Slav...,https://dq06ugkuram52.cloudfront.net/files/635...
2,6380781,https://wikirate.org/~6380781,Walk Free Foundation+MSA whistleblowing mechan...,Hall & Woodhouse Limited,1,https://wikirate.org/~6375083,10/19/2020\n\nSlavery | Hall and Woodhouse\n\n...,https://dq06ugkuram52.cloudfront.net/files/637...
3,6364508,https://wikirate.org/~6364508,Walk Free Foundation+MSA whistleblowing mechan...,Greene King,1,https://wikirate.org/~6357513,Greene King | MODERN SLAVERY STATEMENT\n\nhttp...,https://dq06ugkuram52.cloudfront.net/files/635...
4,6379525,https://wikirate.org/~6379525,Walk Free Foundation+MSA whistleblowing mechan...,Charles Wells,1,https://wikirate.org/~6354153,Charles Wells Ltd \n \nModern Slavery and Huma...,https://dq06ugkuram52.cloudfront.net/files/635...
...,...,...,...,...,...,...,...,...
1713,7311793,https://wikirate.org/~7311793,Walk Free Foundation+MSA whistleblowing mechan...,ECO Animal Health Group Plc,0,https://wikirate.org/~7307585,\n\n \nModern Slavery Act 2015 Statement \n...,https://dq06ugkuram52.cloudfront.net/files/730...
1714,7310878,https://wikirate.org/~7310878,Walk Free Foundation+MSA whistleblowing mechan...,Victoria Plum Limited,0,https://wikirate.org/~7307593,Upto 60% off + 20% off ends\n\n \n\n \n\nSearc...,https://dq06ugkuram52.cloudfront.net/files/730...
1715,7310802,https://wikirate.org/~7310802,Walk Free Foundation+MSA whistleblowing mechan...,Creagh Concrete Products Limited,1,https://wikirate.org/~7307608,\n\nSlavery and Human Trafficking Transparenc...,https://dq06ugkuram52.cloudfront.net/files/730...
1716,7311831,https://wikirate.org/~7311831,Walk Free Foundation+MSA whistleblowing mechan...,Reward Gateway (UK) Ltd,1,https://wikirate.org/~7307616,Reward Gateway Slavery\nand Human Trafficking\...,https://dq06ugkuram52.cloudfront.net/files/730...


In [ ]:
short_list_1500=[]
for i in range(len(df_csv_joined['Text'])):
    length = len(df_csv_joined['Text'][i])
    if length<2000:
      short_list_1500.append(i)

len(short_list_1500) 

115

In [ ]:
df_csv_joined1 = df_csv_joined.drop(short_list_1500)
df_csv_joined1

,Answer ID,Answer Page,Metric,Company,Value,Source Page,Text,URL_link
0,6382834,https://wikirate.org/~6382834,Walk Free Foundation+MSA whistleblowing mechan...,S A Brain Company Ltd,1,https://wikirate.org/~6375302,10/19/2020\n\nSlavery and Human trafﬁcking | S...,https://dq06ugkuram52.cloudfront.net/files/637...
1,6368402,https://wikirate.org/~6368402,Walk Free Foundation+MSA whistleblowing mechan...,Pension Protection Fund,1,https://wikirate.org/~6355100,\n\n \n\n \n\n \n\n \n\n \n\n \n\nModern Slav...,https://dq06ugkuram52.cloudfront.net/files/635...
2,6380781,https://wikirate.org/~6380781,Walk Free Foundation+MSA whistleblowing mechan...,Hall & Woodhouse Limited,1,https://wikirate.org/~6375083,10/19/2020\n\nSlavery | Hall and Woodhouse\n\n...,https://dq06ugkuram52.cloudfront.net/files/637...
3,6364508,https://wikirate.org/~6364508,Walk Free Foundation+MSA whistleblowing mechan...,Greene King,1,https://wikirate.org/~6357513,Greene King | MODERN SLAVERY STATEMENT\n\nhttp...,https://dq06ugkuram52.cloudfront.net/files/635...
4,6379525,https://wikirate.org/~6379525,Walk Free Foundation+MSA whistleblowing mechan...,Charles Wells,1,https://wikirate.org/~6354153,Charles Wells Ltd \n \nModern Slavery and Huma...,https://dq06ugkuram52.cloudfront.net/files/635...
...,...,...,...,...,...,...,...,...
1713,7311793,https://wikirate.org/~7311793,Walk Free Foundation+MSA whistleblowing mechan...,ECO Animal Health Group Plc,0,https://wikirate.org/~7307585,\n\n \nModern Slavery Act 2015 Statement \n...,https://dq06ugkuram52.cloudfront.net/files/730...
1714,7310878,https://wikirate.org/~7310878,Walk Free Foundation+MSA whistleblowing mechan...,Victoria Plum Limited,0,https://wikirate.org/~7307593,Upto 60% off + 20% off ends\n\n \n\n \n\nSearc...,https://dq06ugkuram52.cloudfront.net/files/730...
1715,7310802,https://wikirate.org/~7310802,Walk Free Foundation+MSA whistleblowing mechan...,Creagh Concrete Products Limited,1,https://wikirate.org/~7307608,\n\nSlavery and Human Trafficking Transparenc...,https://dq06ugkuram52.cloudfront.net/files/730...
1716,7311831,https://wikirate.org/~7311831,Walk Free Foundation+MSA whistleblowing mechan...,Reward Gateway (UK) Ltd,1,https://wikirate.org/~7307616,Reward Gateway Slavery\nand Human Trafficking\...,https://dq06ugkuram52.cloudfront.net/files/730...


In [ ]:
stem_sentence_corpus = torch.load('drive/MyDrive/MSR_data/stem_sentence_corpus')

stem_sentence_corpus_pd = pd.Series(stem_sentence_corpus)
stem_sentence_corpus_pd

0       [10/19/2020 slaveri and human trafﬁck | s.a., ...
1       [modern slaveri act statement updated: septemb...
2       [10/19/2020 slaveri | hall and woodhous modern...
3       [green king | modern slaveri statement https:/...
4       [charl well ltd modern slaveri and human traff...
                              ...                        
1713    [modern slaveri act 2015 statement our polici ...
1714    [upto 60% off + 20% off end search for product...
1715    [slaveri and human traffick transpar statement...
1716    [reward gateway slaveri and human traffick sta...
1717    [modern slaveri act statement – januari 2021 o...
Length: 1718, dtype: object

In [ ]:
stem_sentence_corpus_pd.drop(short_list_1500)

0       [10/19/2020 slaveri and human trafﬁck | s.a., ...
1       [modern slaveri act statement updated: septemb...
2       [10/19/2020 slaveri | hall and woodhous modern...
3       [green king | modern slaveri statement https:/...
4       [charl well ltd modern slaveri and human traff...
                              ...                        
1713    [modern slaveri act 2015 statement our polici ...
1714    [upto 60% off + 20% off end search for product...
1715    [slaveri and human traffick transpar statement...
1716    [reward gateway slaveri and human traffick sta...
1717    [modern slaveri act statement – januari 2021 o...
Length: 1603, dtype: object

In [ ]:
data_tensor = torch.load('drive/MyDrive/MSR_data/data_tensor_fnl')
data_tensor_np = data_tensor.detach().numpy()

In [ ]:
data_tensor_np1 = np.delete(data_tensor_np, short_list_1500, axis=0)
data_tensor_np1.shape

(1603, 200, 768)

In [ ]:
data_tensor2 = torch.tensor(data_tensor_np1)
data_tensor2.size()

torch.Size([1603, 200, 768])

#### I Tensor

In [ ]:
I_tensor1 = torch.load('drive/MyDrive/MSR_data/I_tensor')
I_tensor1_np = I_tensor1.detach().numpy()
I_tensor1_np1 = np.delete(I_tensor1_np, short_list_1500, axis=0)
I_tensor1_np1.shape

(1603, 1, 200)

In [ ]:
I_tensor2 = torch.tensor(I_tensor1_np1)
I_tensor2.size()

torch.Size([1603, 1, 200])

### Model training

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
label = df_csv_joined1['Value'].tolist()
len(label)

1603

In [ ]:
label = df_csv_joined1['Value'].tolist()

train_x = data_tensor2[:1200]
train_x = train_x.permute(0,2,1)
train_y = torch.tensor(label[:1200])
train_i = I_tensor2[:1200]

val_x = data_tensor2[1200:]
val_x = val_x.permute(0,2,1)
val_y = torch.tensor(label[1200:])
val_i = I_tensor2[1200:]

test_x = data_tensor2[1200:]
test_x = test_x.permute(0,2,1)
test_y = torch.tensor(label[1200:])
test_i = I_tensor2[1200:]

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 50

train_data = TensorDataset(train_x, train_y, train_i)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler = train_sampler, batch_size=batch_size, shuffle=False)

val_data = TensorDataset(val_x, val_y, val_i)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size, shuffle=False)

In [ ]:
class model(nn.Module):

    def __init__(self):
        
        super(model, self).__init__()
                        
        self.conv1 = nn.Conv1d(in_channels = 768, out_channels=1, kernel_size=1, stride=1, padding=0)
        
        self.conv2 = nn.Conv1d(in_channels = 768, out_channels=1024, kernel_size=1, stride=1, padding=0)
        
        self.fc1 = nn.Linear(1024, 100)
        
        self.fc2 = nn.Linear(100, 2)
                
        self.softmax1 = nn.Softmax(dim=-1)
        
        self.softmax2 = nn.Softmax(dim=1)
        
        self.relu =  nn.ReLU()
        
        self.dropout = nn.Dropout(0.1)
        

    #define the forward pass
    def forward(self, x, y, i):
        
        lbda = 25
                        
        alpha = self.conv1(x) #(btch_sz, 768, 200)-> (btch_sz, 1, 200)
        
        x_2 = self.conv2(x)
        
        x_2 = self.relu(x_2)
                                                        
        alpha1 = alpha + lbda * i #(btch_sz, 1, 200) + (btch_sz,1, 200)*(btch_sz, 1, 200) 
                                
        alpha_prime = self.softmax1(alpha1) #(btch_sz, 1, 200)
                        
        alpha_prime_permute = alpha_prime.permute(0,2,1) #(btch_sz, 200, 1) 
                                        
        h = torch.matmul(x_2, alpha_prime_permute)  #(btch_sz, 768, 200)*(btch_sz, 200, 1) = (btch_sz, 768, 1)
                                
        h = h.view(h.shape[0], -1) #(btch_sz, 768)
                        
        output = self.relu(self.fc1(h)) #(btch_sz, 768, 512) 
        
        output = self.dropout(output)
        
        output = self.fc2(output)
        
        output = self.softmax2(output)
                                 
        return output

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
 
class_wts = compute_class_weight('balanced', classes= torch.unique(train_y).tolist(), y=train_y.tolist())

print(class_wts)

weights = torch.tensor(class_wts, dtype = torch.float)
weights = weights.to(device)


[1.05820106 0.9478673 ]


In [ ]:
from torch import optim

Model = model()

Model = Model.to(device)

cross_entropy = nn.CrossEntropyLoss(weight=weights)

from transformers import AdamW

optimizer = optim.AdamW(Model.parameters(), lr = 0.001, weight_decay=1e-5)

epochs = 100


In [ ]:
def train():
    Model.train()
    total_loss, total_accuracy = 0, 0

    train_correct = 0
    train_total = 0
    
    for step, batch in enumerate(train_dataloader):

        batch = [r.to(device) for r in batch]
 
        x, y, i = batch

        Model.zero_grad()        

        preds = Model(x, y, i)

        loss = cross_entropy(preds, y)

        total_loss = total_loss + loss.item()
     
        loss.backward()

        torch.nn.utils.clip_grad_norm_(Model.parameters(), 1.0)  # clip the the gradients to 1.0, preventing the exploding gradient problem

        optimizer.step()  # update parameters

        _, predicted = torch.max(preds.data, 1)
        train_total += y.size(0)
        train_correct += (predicted == y).sum().item()

        preds=preds.detach().cpu().numpy()   # if model predictions are stored on GPU, push it to CPU
        
    avg_loss = total_loss / len(train_dataloader) # compute the training loss of the epoch
    train_acc = train_correct/train_total
  
    return avg_loss, train_acc #returns train loss and predictions


In [ ]:
def evaluate():
    
    print("\nEvaluating...")
    Model.eval()   # deactivate dropout layers
    total_loss = 0

    val_total = 0
    val_correct = 0

    for step,batch in enumerate(val_dataloader):   # iterate over batches
        
        batch = [t.to(device) for t in batch] # push the batch to gpu

        x, y, i = batch
        
        with torch.no_grad(): # deactivate autograd
            
            preds = Model(x, y, i)
            
            loss = cross_entropy(preds,y)

            total_loss = total_loss + loss.item()

            _, predicted = torch.max(preds.data, 1)
            val_total += y.size(0)
            val_correct += (predicted == y).sum().item()

            preds = preds.detach().cpu().numpy()
            
    avg_loss = total_loss / len(val_dataloader) 
    val_acc = val_correct/val_total

    return avg_loss, val_acc

In [ ]:
import numpy as np
best_valid_loss = float('inf')  # set initial loss to infinite

# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]
train_acces = []
val_acces = []

for epoch in range(epochs): #for each epoch
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))

    train_loss, train_acc = train() #train model
    valid_loss, val_acc = evaluate() #evaluate model
    
    if valid_loss < best_valid_loss: #save the best model
        best_valid_loss = valid_loss
        torch.save(Model.state_dict(), 'saved_weights.pt')
    
    # append training and validation loss
    
    train_acces.append(train_acc)
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    val_acces.append(val_acces)
    
    print('Train acc', np.round(train_acc, 3), 'Val acc', np.round(val_acc, 3))
    print('Train loss', np.round(train_loss, 3), 'Val loss', np.round(valid_loss, 3))



 Epoch 1 / 100

Evaluating...
Train acc 0.652 Val acc 0.68
Train loss 0.671 Val loss 0.647

 Epoch 2 / 100

Evaluating...
Train acc 0.665 Val acc 0.68
Train loss 0.644 Val loss 0.65

 Epoch 3 / 100

Evaluating...
Train acc 0.668 Val acc 0.521
Train loss 0.631 Val loss 0.683

 Epoch 4 / 100

Evaluating...
Train acc 0.664 Val acc 0.64
Train loss 0.626 Val loss 0.661

 Epoch 5 / 100

Evaluating...
Train acc 0.668 Val acc 0.68
Train loss 0.623 Val loss 0.638

 Epoch 6 / 100

Evaluating...
Train acc 0.686 Val acc 0.663
Train loss 0.616 Val loss 0.642

 Epoch 7 / 100

Evaluating...
Train acc 0.688 Val acc 0.635
Train loss 0.617 Val loss 0.651

 Epoch 8 / 100

Evaluating...
Train acc 0.654 Val acc 0.672
Train loss 0.618 Val loss 0.637

 Epoch 9 / 100

Evaluating...
Train acc 0.676 Val acc 0.675
Train loss 0.62 Val loss 0.64

 Epoch 10 / 100

Evaluating...
Train acc 0.672 Val acc 0.558
Train loss 0.615 Val loss 0.68

 Epoch 11 / 100

Evaluating...
Train acc 0.682 Val acc 0.603
Train loss 0.61

In [ ]:
path = 'saved_weights.pt'
Model.load_state_dict(torch.load(path))

print('Train_report:')
with torch.no_grad():
  train_preds = Model(train_x.to(device), train_y.to(device), train_i.to(device))
  train_preds = train_preds.detach().cpu().numpy()

train_preds = np.argmax(train_preds, axis = 1)
print(classification_report(train_y, train_preds))


print('Val_report:')
with torch.no_grad():
  val_preds = Model(val_x.to(device), val_y.to(device), val_i.to(device))
  val_preds = val_preds.detach().cpu().numpy()

val_preds = np.argmax(val_preds, axis = 1)
print(classification_report(val_y, val_preds))

print('Test_report:')
with torch.no_grad():
  test_preds = Model(test_x.to(device), test_y.to(device), test_i.to(device))
  test_preds = test_preds.detach().cpu().numpy()

test_preds = np.argmax(test_preds, axis = 1)
print(classification_report(test_y, test_preds))

Train_report:
              precision    recall  f1-score   support

           0       0.77      0.49      0.60       567
           1       0.66      0.87      0.75       633

    accuracy                           0.69      1200
   macro avg       0.71      0.68      0.68      1200
weighted avg       0.71      0.69      0.68      1200

Val_report:
              precision    recall  f1-score   support

           0       0.66      0.31      0.42       150
           1       0.69      0.91      0.78       253

    accuracy                           0.68       403
   macro avg       0.67      0.61      0.60       403
weighted avg       0.68      0.68      0.65       403

Test_report:
              precision    recall  f1-score   support

           0       0.66      0.31      0.42       150
           1       0.69      0.91      0.78       253

    accuracy                           0.68       403
   macro avg       0.67      0.61      0.60       403
weighted avg       0.68      0.68  

In [ ]:
x = list(range(0,100))
plt.figure(figsize=(10,6))
plt.plot(x, train_acces)
plt.plot(x, val_acces)

plt.xlabel('epochs')
plt.ylabel('Accuracy')
plt.title('Model train and validation loss')